<a href="https://colab.research.google.com/github/oleoespindola/RedditSent/blob/main/app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <img src="https://media.giphy.com/media/EfDzi9nKLFPtF0VwTk/giphy.gif?cid=790b7611oh7utjjmlaa2v5p8nz9285pq24nl7jp0kgd6jyis&ep=v1_stickers_search&rid=giphy.gif&ct=s" alt="Giphy: Mascote do Reddit erguendo um corçação"  style="align=center" height="50"/> Redddit Sent

Análise de Sentimentos de Submissões do Reddit

Este script desenvolvido tem como objetivo automatizar a análise de sentimentos em comentários de submissões do Reddit. Ele utiliza a API do Reddit para coletar submissões de diferentes subreddits especificados pelo usuário. Em seguida, solicita a classificação dos comentários em positivos, negativos ou neutros em relação às emoções humanas por meio da integração com o ChatGPT, uma inteligência artificial de linguagem natural.

\
___
\

**Funcionalidades Principais**

1.  **Coleta de Dados do Reddit**: O script permite especificar os subreddits alvo para coletar submissões recentes.

2. **Análise de Sentimentos**: Utilizando a API do ChatGPT, o script classifica os comentários associados às submissões em três categorias: positivos, negativos ou neutros.

3. **Construção de Análise**: Com base nos resultados da classificação de sentimentos, o script gera uma análise detalhada dos dados coletados. Isso pode incluir estatísticas sobre a distribuição de sentimentos, palavras-chave associadas a cada categoria e insights sobre a percepção geral dos usuários em relação aos tópicos discutidos nos subreddits selecionados.

#### Bibliotecas e Importações Gerais

In [4]:
from google.colab import userdata

import pandas as pd

### Reddit

Coleta de dados no Reddit, baseada em um assunto pré-determinado, limitando-se ou não a um subreddit. Além disso, nesta seção o tamanhao da amostra é determinado.

#### Bibliotecas e Importações

In [5]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 3.9 MB/s eta 0:00:00


In [6]:
import praw

#### Variáveis do Reddit
- Define o assunto da pesquisa;
- Limita os subreddits;
- limita o tamanho da amostra.

In [42]:
query = 'UNIP'
subreddit = 'all'  # Pesquisar em todos os subreddits
limit = 5 # Limita a quantidade de resultados

**Atribuição de Credenciais**

In [8]:
reddit = praw.Reddit(
    client_id = userdata.get('reddit_client_id'),
    client_secret = userdata.get('reddit_client_secret'),
    user_agent = userdata.get('reddit_user_agent')
)

#### Coleta e Tratamento dos daddos

**1. Coleta**

In [43]:
subreddit = reddit.subreddit(subreddit)
submissions  = subreddit.search(query, limit=limit)

**2. Tratamento dos Dados**

In [44]:
data = {
    "title": [],
    "score": [],
    "id": [],
    "url": [],
    "created": [],
    "author": []
}

for submission in submissions:
    data["title"].append(submission.title)
    data["score"].append(submission.score)
    data["id"].append(submission.id)
    data["url"].append(submission.url)
    data["created"].append(submission.created)
    data["author"].append(submission.author)

df = pd.DataFrame(data)

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.



**BackUp da Consulta**

In [11]:
df.to_csv('data.csv', index=False)

### OpenAI

Envia os *submissions* (postagens) coletados anteriormente ao chatGPT e pele para classifica-los como positivos, negativosm ou neutros.

#### Bibliotecas e Importações

In [12]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.5 MB/s eta 0:00:00


In [13]:
from openai import OpenAI

**Atribuição de Credenciais**

In [14]:
client = OpenAI(
    api_key = userdata.get('openai_api_key')
)

#### Envio e Coleta das Informações

**1. Função de Classificação das submissions**

Cria uma função para adicionar as respostas da AI em uma nova coluna do DataFrame

In [142]:
def classify_submission(submission):
  completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "O chatbot responde com uma das três pavras seguintes: positivo, negativo, neutro. A palavra escolhida serve para adjetivar o sentimento humano expresso na mensagem do usuário. O chatbot nunca deve responder em inglês ou usar variações de letras maiúsculas, minúsculas ou de potuação. Além disso, as respostas serão sempre em português (brasileiro) e apenas uma palavra das três deve ser eescolhida."},
    {"role": "user", "content": f"{submission}"}
  ]
)
  return completion.choices[0].message.content

**2. Aplicação das classificações**

In [143]:
df['submission_rating'] = df['title'].apply(classify_submission)

**BackUp da Consulta**

In [144]:
df.to_csv('data.csv', index=False)

### Data Analysis

**Resultados Obtidos**

In [147]:
df[['title', 'score', 'submission_rating']]

,title,score,submission_rating
0,Como funciona essa faculdade Unip ?,11,neutro
1,unip e etc é fim de carreira?,8,negativo
2,Unip é uma boa escolha?,6,positivo
3,Graduação na UNIP vale a pena? (Desabafo),8,positivo
4,A unip é boa?,6,neutro
